# Running modelx with ipyparallel

This notebook demonstrates how to run modelx models in parallel using 
[ipyparallel](https://ipyparallel.readthedocs.io/en/latest/).

This example:

* launches 10 IPython engines,
* loads 100,000 model points,
* Send a block of 10,000 model points to each engine,
* runs modelx with the 10,000 model points on each engine and
* get results from all the engines and concatinate them.


This notebook consumes about 30-40GB memory, and may crash on a machine with less memory. In such case, consider reducing the number of model points to test.

This notebook uses ipyparallel 8.2.0, and may not work with older versions of ipyparallel.
As of March 12, 2022, ipyparallel 8.2.0 is available on [conda-forge](https://anaconda.org/conda-forge/ipyparallel/), but not in [the anaconda package](https://docs.anaconda.com/anaconda/packages/py3.9_win-64/).
So if you're using anaconda, update ipyparallel by `conda install ipyparallel=8.2.0`.

 

Start a cluster with 10 engines.`rc` is a `Client` object.

In [1]:
import ipyparallel
ipp = ipyparallel
cluster = ipp.Cluster(n=10)
rc = cluster.start_and_connect_sync()

Starting 10 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/10 [00:00<?, ?engine/s]

Read the entire model point table into a DataFrame in this process.  The table has 100,000 model points.

In [2]:
import pandas as pd
model_point_all =  pd.read_excel('model_point_table_100K.xlsx', index_col=0)
model_point_all

,spec_id,age_at_entry,sex,policy_term,policy_count,sum_assured,duration_mth,premium_pp,av_pp_init
policy_id,,,,,,,,,
1,B,47,M,20,62,804000,0,804000,0
2,C,29,F,9999,26,519000,0,900,0
3,B,51,F,10,44,409000,0,409000,0
4,A,32,M,15,85,128000,0,128000,0
5,B,28,M,20,16,698000,0,698000,0
...,...,...,...,...,...,...,...,...,...
99996,C,21,M,9999,21,152000,0,300,0
99997,A,24,F,15,77,928000,0,928000,0
99998,A,46,F,15,67,662000,0,662000,0


Evenly split the entire table into 10 blocks and send each block to each engine.
The block of model points on each engine is assigned to a global variable named `model_point`.

In [3]:
for i in range(10):
    rc[i]['model_point'] = model_point_all.iloc[i * 10000: (i+1) * 10000]

Send the code to execute to each block.
The code:
* imports `modelx`
* loads the sample model and assigns it to `m`
* Calculate `result_pv()` and assigns the result to `result`

In [5]:
code = """
import modelx as mx
m = mx.read_model('CashValue_ME')
m.Projection.model_point_table = model_point
result = m.Projection.result_pv()
"""
rc[:].execute(code, block=True)

<AsyncResult(execute): finished>

Get the results back from all the engines and concatinate into one DataFrame.

In [6]:
result = pd.concat(rc[:]['result']) 
result

,Premiums,Death,Surrender,Maturity,Expenses,Commissions,Investment Income,Change in AV,Net Cashflow
policy_id,,,,,,,,,
1,4.984800e+07,1.279624e+06,2.119151e+07,2.648411e+07,7.186362e+05,2.492400e+06,1.568053e+07,8.705435e+06,4.656817e+06
2,5.992953e+06,1.871726e+06,2.842599e+06,0.000000e+00,4.757631e+05,2.996477e+05,2.804023e+06,2.093446e+06,1.213795e+06
3,1.799600e+07,2.118581e+05,5.513367e+06,1.140103e+07,3.867355e+05,8.998000e+05,2.408106e+06,1.495734e+06,4.955837e+05
4,1.088000e+07,7.187407e+04,3.874992e+06,5.803388e+06,8.770947e+05,5.440000e+05,2.298573e+06,1.308987e+06,6.982380e+05
5,1.116800e+07,9.580780e+04,4.775143e+06,6.102210e+06,1.863451e+05,5.584000e+05,3.551421e+06,1.970349e+06,1.031167e+06
...,...,...,...,...,...,...,...,...,...
99996,1.659954e+06,3.911611e+05,8.409576e+05,0.000000e+00,3.959725e+05,8.299768e+04,8.236990e+05,6.756107e+05,9.695319e+04
99997,7.145600e+07,3.429920e+05,2.546623e+07,3.822980e+07,7.949496e+05,3.572800e+06,1.511732e+07,8.608484e+06,9.558065e+06
99998,4.435400e+07,6.358990e+05,1.575364e+07,2.335160e+07,6.898746e+05,2.217700e+06,9.315364e+06,5.306157e+06,5.714497e+06


Clear all calculated values on all the engines. 

In [ ]:
rc[:].execute("m.clear_all()", block=True)